k<center>
<h1>Welcome to the Lab 🥼🧪</h1>
</center>

## How can I analyze the behavior and performance of major single family rental (SFR) entities across their portfolios using the Parcl Labs API?

This notebook demonstrates how to use the Parcl Labs API to conduct in-depth analysis of large-scale SFR operators. By leveraging our entity-to-unit mapping and housing event data, you'll learn to:

- Query and aggregate property data for specific SFR entities
- Track portfolio changes through acquisition and disposition events
- Analyze rental trends and performance metrics at scale
- Implement comparative analysis between major market players

We'll focus on the Phoenix metro market and two prominent SFR operators, but the methodology is applicable across Parcl Labs’ 70,000+ available markets and all entities in our [events history endpoint](https://docs.parcllabs.com/reference/property).

### What you will build
In this notebook, we will:

- Quantify entity-specific inventory
- Calculate time-series data for acquisitions and dispositions
- Extract and analyze rental event data
- Compute and visualize key performance indicators
<p align="center">
  <img src="../../../images/sfh_ownership_for_individual_entities.png" alt="Alt text">
</p>

| Quarter | Total Acquisitions | Total Dispositions | Entity | Net  |
|---------|--------------------|--------------------|--------|------|
| 2020Q4  | 32.0               | 1.0                | AMH    | 31.0 |
| 2021Q1  | 46.0               | 0.0                | AMH    | 46.0 |
| 2021Q2  | 54.0               | 0.0                | AMH    | 54.0 |
| 2021Q3  | 25.0               | 0.0                | AMH    | 25.0 |
| 2021Q4  | 37.0               | 1.0                | AMH    | 36.0 |
| 2022Q1  | 57.0               | 0.0                | AMH    | 57.0 |
| 2022Q2  | 24.0               | 0.0                | AMH    | 24.0 |
| 2022Q3  | 19.0               | 0.0                | AMH    | 19.0 |
| 2022Q4  | 31.0               | 0.0                | AMH    | 31.0 |
| 2023Q2  | 28.0               | 0.0                | AMH    | 28.0 |

<p align="center">
  <img src="../../../images/acquisitions_vs_dispositions_amh_phoenix.png" alt="Alt text">
</p>

<p align="center">
  <img src="../../../images/acquisitions_vs_dispositions_tricon_phoenix.png" alt="Alt text">
</p>
<p align="center">
  <img src="../../../images/entity_monthly_rental_price_phoenix.png" alt="Alt text">
</p>

<p align="center">
  <img src="../../../images/entity_total_rental_events_phoenix.png" alt="Alt text">
</p>


#### Need help getting started?

As a reminder, you can get your Parcl Labs API key [here](https://dashboard.parcllabs.com/login) to follow along. 

Please note that you will need a paid account to access the full functionality demonstrated in this notebook. You can easily upgrade directly via your [API dashboard](https://dashboard.parcllabs.com/login).

To run this notebook immediately, you can use Google Colab. 

Remember, you must set your `PARCL_LABS_API_KEY` in the Colab environment before executing the code.
Run in Colab --> [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ParclLabs/parcllabs-cookbook/blob/main/examples/housing_market_research/investor_analytics/entity_analysis.ipynb)


In [ ]:
# if needed, install and/or upgrade to the latest verison of the Parcl Labs Python library
%pip install --upgrade parcllabs

After installing the required libraries, we need to load them and instantiate the Parcl Labs client. The client is a Python library designed to facilitate and optimize the user experience with the Parcl Labs API. It handles searching, retrieving, and formatting the data for us. 

To use the client, you need to have an `API_KEY`, which is available in your [dashboard](https://dashboard.parcllabs.com/). While you can enter your `API_KEY` directly, it is recommended to save it as an environment variable for better security. If you are using Colab, you can follow these [steps](https://medium.com/@parthdasawant/how-to-use-secrets-in-google-colab-450c38e3ec75).


In [2]:
# Environment setup
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from parcllabs import ParclLabsClient
from parcllabs.beta.charting.utils import (
    create_labs_logo_dict,
    save_figure,
    sort_chart_data
    )
from parcllabs.beta.charting.styling import default_style_config as style_config 
from parcllabs.beta.charting.styling import SIZE_CONFIG
from parcllabs import ParclLabsClient
from plotly.subplots import make_subplots


# Create a Parcl Labs client
client = ParclLabsClient(
    api_key=os.environ.get('PARCL_LABS_API_KEY', "<your Parcl Labs API key if not set as environment variable>"), 
    limit=10 # set default limit
)

We will analyze the Phoenix metro market. To find the corresponding `parcl_id` for that market, we can use the `search.markets.retrieve` method of the client with the appropriate parameters. In this case, the main parameter defines the type of market we are looking for. For the Phoenix metro area, we set this value to "CBSA," which is an abbreviation for Core-Based Statistical Area, the official term used by the Census Bureau.


In [ ]:
# Search for a specific market by name and type
# In this case, we are going to search for Phoenix CBSA (Core Based Statistical Area)
market = client.search.markets.retrieve(
    query='phoenix',
    location_type='CBSA',
)

# Get the name and parcl_id of the market
market_name = market["name"].iloc[0]
market_parcl_id = market["parcl_id"].iloc[0]
print(f' The name of the market is {market_name} and the parcl_id is {market_parcl_id}')

In this notebook, we will identify properties owned by two different large SFR (Single-Family Rental) operators in Phoenix. To do this, we first need to identify which single-family homes they currently own using the `property.search.retrieve` method of the client. 

In a previous notebook, we covered all the details on how to narrow your search using the available parameters for this endpoint. You can find that notebook [here](https://github.com/ParclLabs/parcllabs-cookbook/blob/main/examples/getting_started/property_data_download.ipynb) if you need a refresher.

For this example, we will focus on `Tricon` and `American Homes 4 Rent (AMH)`, two of the largest housing rental companies in the country. We will look at the Phoenix Metropolitan area (`parcl_id: 2900245`), one of the epicenters of institutional activity for large portfolios, and analyze their behavior in this market. We will start by identifying single-family homes currently owned by AMH, specifying the required parameters: `parcl_id`, `property_type`, and `current_entity_owner_name`. You will notice we have several commented-out parameters that could be used to further narrow our search. Feel free to uncomment and tailor the search to suit your needs.


In [ ]:

# Define the search parameters
search_params_amh = {
    'parcl_ids': [2900245],  # Required
    'property_type': 'SINGLE_FAMILY',  # Required
    'current_entity_owner_name': 'AMH',  # Specify one of the options or None
    #'square_footage_min': 1000,
    #'square_footage_max': 5000,
    #'bedrooms_min': 2,
    #'bedrooms_max': 4,
    #'bathrooms_min': 2,
    #'bathrooms_max': 3,
    #'year_built_min': 1990,
    #'year_built_max': 2023,
    #'event_history_sale_flag': True,
    #'event_history_rental_flag': False,
    #'event_history_listing_flag': True,
    #'current_new_construction_flag': False,
    #'current_owner_occupied_flag': True,
    #'current_investor_owned_flag': False
}

# We search for properties in the market we defined above using the parameters that are not commented out.
# We can pass the search_params dictionary to the retrieve method to get the search results using **search_params
amh_homes_phoenix = client.property.search.retrieve(**search_params_amh)

print(f"Found {len(amh_homes_phoenix)} properties matching the criteria.")

We repeat the same approach to get single-family homes owned by `Tricon`.

In [ ]:
# now repeat for Tricon
# Define the search parameters
search_params_tricon = {
    'parcl_ids': [2900245],  # Required
    'property_type': 'SINGLE_FAMILY',  # Required
    'current_entity_owner_name': 'TRICON',  # Specify one of the options or None
    #'square_footage_min': 1000,
    #'square_footage_max': 5000,
    #'bedrooms_min': 2,
    #'bedrooms_max': 4,
    #'bathrooms_min': 2,
    #'bathrooms_max': 3,
    #'year_built_min': 1990,
    #'year_built_max': 2023,
    #'event_history_sale_flag': True,
    #'event_history_rental_flag': False,
    #'event_history_listing_flag': True,
    #'current_new_construction_flag': False,
    #'current_owner_occupied_flag': True,
    #'current_investor_owned_flag': False
}

# We search for properties in the market we defined above using the parameters that are not commented out.
# We can pass the search_params dictionary to the retrieve method to get the search results using **search_params
tricon_homes_phoenix = client.property.search.retrieve(**search_params_tricon)

print(f"Found {len(tricon_homes_phoenix)} properties matching the criteria.")

Now we can visualize the total number of properties owned by each entity in the market. We will use a bar chart to visualize the data.

In [ ]:
# Concatenate the dataframes and group by the entity owner name
total_home_stock = pd.concat([amh_homes_phoenix, tricon_homes_phoenix])

# Group by 'current_entity_owner_name' to calculate the total number of units per entity
df_melted = (total_home_stock
             .groupby('current_entity_owner_name').size()
             .reset_index(name='Total Units')  # Add the total units column
             .rename(columns={'current_entity_owner_name': 'Entity_PF'})  # Rename the entity column for readability
             .sort_values('Total Units', ascending=False)  # Sort by total units in descending order
             )

# Define custom colors for the chart
colors = ['#142872', '#8a9cb7']  # Ensure these colors are distinct and accessible

# Create the stacked bar chart using Plotly Express
fig = px.bar(df_melted, 
             x='Total Units', 
             y='Entity_PF', 
             barmode='relative',  # Use relative barmode for better comparison
             title=f'Total Units by Entity {market_name}',  # Correct the title text
             color='Entity_PF',  # Differentiate entities by color
             color_discrete_sequence=colors  # Apply custom colors
)

# Update trace properties
fig.update_traces(marker=dict(line=dict(width=0)))  # Remove border lines from the bars

# Customize the layout of the chart
fig.update_layout(
    margin=dict(l=40, r=40, t=100, b=40),  # Adjust margins for better fitting
    showlegend=False,  # Hide the legend for simplicity
    title={
        'text': f'Total Units Owned by Large Entities in {market_name}'.upper(),  # Updated title for clarity
        'y': 0.95,
        'x': 0.5,
        'xanchor': 'center',  # Center the title
        'yanchor': 'top',
        'font': style_config['title_font']  # Apply the custom font from style_config
    },
    xaxis=dict(
        title_text='',  # No label for x-axis since it's self-explanatory
        tickformat=',',  # Format large numbers with commas
        showgrid=style_config['showgrid'],  # Grid visibility based on style_config
        gridwidth=style_config['gridwidth'],  # Grid width from style_config
        gridcolor=style_config['grid_color'],  # Grid color from style_config
        linecolor=style_config['line_color_axis'],  # Axis line color
        linewidth=style_config['linewidth'],  # Axis line width
        titlefont=style_config['title_font_axis'],  # Axis title font from style_config
        tickfont=dict(size=style_config['axis_font']['size'], color=style_config['axis_font']['color'])  # Tick font style
    ),
    yaxis=dict(
        title_text='Total Units in Portfolio',  # Y-axis title
        showgrid=style_config['showgrid'],  # Grid visibility for y-axis
        gridwidth=style_config['gridwidth'],  # Grid width for y-axis
        gridcolor=style_config['grid_color'],  # Grid color for y-axis
        tickfont=style_config['axis_font'],  # Tick font for y-axis
        zeroline=False,  # Hide the zero line for cleaner visuals
        tickformat=',',  # Format y-axis numbers with commas
        linecolor=style_config['line_color_axis'],  # Y-axis line color
        linewidth=style_config['linewidth'],  # Y-axis line width
        titlefont=style_config['title_font_axis']  # Y-axis title font
    ),
    plot_bgcolor=style_config['background_color'],  # Background color from style_config
    paper_bgcolor=style_config['background_color'],  # Paper color to match the chart's background
    font=dict(color=style_config['font_color']),  # Set font color for the entire chart
    legend_title_text='',  # Hide legend title as it's redundant
    autosize=False,  # Disable auto-sizing to control dimensions manually
    width=1600,  # Set chart width
    height=800,  # Set chart height
    title_font=dict(size=24),  # Set font size for the title
    xaxis_title_font=dict(size=18),  # Font size for x-axis title
    yaxis_title_font=dict(size=18),  # Font size for y-axis title
    legend_title_font=dict(size=14),  # Font size for legend title (though it's hidden)
    legend_font=dict(size=12),  # Font size for legend items
    legend=dict(
        x=style_config['legend_x'],  # X position for the legend
        y=style_config['legend_y'],  # Y position for the legend
        xanchor=style_config['legend_xanchor'],  # X anchor for the legend position
        yanchor=style_config['legend_yanchor'],  # Y anchor for the legend position
        font=style_config['legend_font'],  # Font settings for the legend
        bgcolor='rgba(0, 0, 0, 0)'  # Transparent background for the legend
    )
)

# Add the Labs logo to the chart
fig.add_layout_image(create_labs_logo_dict())  # Function to add the logo as an image to the chart

# Save the chart to the images folder
fig.write_image('../../../images/sfh_ownership_for_individual_entities.png')  # Make sure the path is correct

# Display the chart
fig.show()


Visualizing the total stock is important and provides a good starting point for analysis, but we can do much more now that we have the `parcl_property_id` of all the properties owned by `AMH` and `Tricon`. We can start uncovering some of their activity by looking at the [property events endpoint](https://docs.parcllabs.com/reference/property_events_v1_property_event_history_post), which contains detailed information about properties, including listings, sales, and rentals.

Let's get that information by putting all the `parcl_property_id` values in a list and passing that to the `property.events.retrieve` method of the client. We will focus on acquisitions and dispositions. To do this, we need to retrieve the full history of the properties currently owned by AMH and Tricon. We will start with AMH.


In [ ]:
# Pass the parcl_property_ids from the search results into a list named search_results_ids_amh to retrieve the sale events 
# for those properties
search_results_ids_amh = amh_homes_phoenix['parcl_property_id'].tolist()

# Define the parameters to use in the search for property events. In this case, we are looking for all sale events 
# related to the properties, so we set the event_type as 'SALE'. We do not define a begin_date or end_date to capture 
# all available sale events.
property_events_parameters_amh = {
    'parcl_property_ids': search_results_ids_amh,  # List of property IDs to search
    'event_type': 'SALE',  # Filter for sale events only
     'start_date': '2019-01-01', # Filter for events starting from this date
}

# Call the client with the list of property IDs and the event_type as 'SALE' to retrieve the sale events
# for these properties. We can pass the parameters dictionary to the retrieve method using **property_events_parameters_amh
sale_events_amh = client.property.events.retrieve(
    **property_events_parameters_amh
)

# Display the total number of sale events found and preview the first two records
print(f"Found {len(sale_events_amh)} events matching the criteria.")
print(sale_events_amh.head(2))  # Preview the first two sale events


In [ ]:
# Repeat the process for Tricon

# Pass the parcl_property_ids from the search results into a list named search_results_ids_tricon 
# to retrieve the sale events for those properties
search_results_ids_tricon = tricon_homes_phoenix['parcl_property_id'].tolist()

# Define the parameters for the property events search. As with AMH, we are looking for all sale events,
# so we set the event_type to 'SALE' and do not specify a begin_date or end_date to capture all sales.
property_events_parameters_tricon = {
    'parcl_property_ids': search_results_ids_tricon,  # List of Tricon property IDs
    'event_type': 'SALE',  # Filter for sale events only
    'start_date': '2019-01-01',  # Filter for events starting from this date
}

# Call the client with the list of property IDs and event_type as 'SALE' to retrieve the sale events for Tricon properties.
# We pass the parameters dictionary using **property_events_parameters_tricon
sale_events_tricon = client.property.events.retrieve(
    **property_events_parameters_tricon
)

# Display the total number of sale events found and preview the first two records
print(f"Found {len(sale_events_tricon)} events matching the criteria.")
print(sale_events_tricon.head(2))  # Preview the first two sale events


The historical event data provides a solid starting point, but we need to modify our dataset to distinguish when one of the entities we are analyzing is purchasing a property and when they are disposing of it. We will utilize the `sale_index` field, which orders sales events chronologically. The overall approach for this analysis is as follows:

* Sort the dataframe containing the sales event history in chronological order.
* Create two new fields, `month` and `quarter`, to facilitate easier aggregation of totals.
* Make a copy of the original dataframe, keeping the same columns, but rename them so we can track the two dataframes separately (you'll see why shortly).
* Merge the copied dataframe with the original one on `parcl_property_id`.
* Filter the merged dataframe to only include rows where the `sale_index` of one dataframe matches the next event in the other (`sale_index` + 1). This links the sales events, identifying the buyer and seller chronologically, and prevents double-counting.
* Apply additional filters to identify when the seller is the entity we are analyzing, and when the buyer is that entity.
* Finally, aggregate the results at a quarterly level to capture all acquisitions and dispositions.

The code below implements these steps, wrapped in a function for easier reuse later in the notebook.

In [9]:
# create the function to return aggregated disposition and acquisitions aggregated by quarter
def process_events(df, entity):
    """
    This function processes sales event data by creating a dataframe that aligns
    seller and buyer transactions, and applies custom logic for flagging disposition 
    and acquisition events based on the provided entity.

    Args:
    df (pd.DataFrame): The dataframe containing sales event data.
    entity (str): The entity name used for filtering and flagging.

    Returns:
    pd.DataFrame: Processed dataframe with buyer and seller events aligned and flags computed.
    """
    # Step 1: Create 'month' and 'quarter' columns based on 'event_date'
    df_events = (
        df
        .assign(
            month=lambda x: pd.to_datetime(x['event_date']).dt.to_period('M'),   # Create month period from 'event_date'
            quarter=lambda x: pd.to_datetime(x['event_date']).dt.to_period('Q')  # Create quarter period from 'event_date'
        )
        .loc[:, [
            'parcl_property_id', 'sale_index', 'event_date', 'price', 
            'event_type', 'entity_owner_name', 'investor_flag', 'month', 'quarter'
        ]]  # Select relevant columns
        .sort_values(['parcl_property_id', 'sale_index'], ascending=[True, True])  # Sort by 'parcl_property_id' and 'sale_index'
    )

    df_events = (
        df
        # Step 1: Create 'month' and 'quarter' columns based on the 'event_date'.
        # This will make it easier to analyze data by time periods.
        .assign(
            month=lambda x: pd.to_datetime(x['event_date']).dt.to_period('M'),
            quarter=lambda x: pd.to_datetime(x['event_date']).dt.to_period('Q')
        )
        .loc[:, ['parcl_property_id', 'sale_index', 'event_date', 'price', 
                 'event_type', 'entity_owner_name', 'investor_flag', 'month', 'quarter'
                 ]]
        .sort_values(['parcl_property_id', 'sale_index'], ascending=[True, True])   
        )
    # Step 2: Prepare the next event data for buyer-side
    next_event_df = (
        df_events
        .pipe(lambda df: (
            df.rename(columns={
                'event_date': 'next_event_date',
                'price': 'next_price',
                'event_type': 'next_event_type',
                'month': 'next_month',
                'quarter': 'next_quarter',
                'sale_index': 'next_sale_index',
                })
            )
            )
        )
    # Step 3: Merge the original dataframe with the next event dataframe
    results = (
        df_events
        .merge(
            next_event_df,                                # Merge seller and buyer data
            on='parcl_property_id',                       # Merge on 'parcl_property_id'
            suffixes=('_seller', '_buyer')                # Use suffixes to distinguish seller and buyer columns
        )
        # Step 4: Filter the cases where the sale event is the next in sequence
        .query('(sale_index == next_sale_index - 1) or (sale_index == 1 and next_sale_index == 1)')
        
        # Step 5: Calculate the number of days between sales events
        .assign(days_between_sales=lambda x: (pd.to_datetime(x['next_event_date']) - pd.to_datetime(x['event_date'])).dt.days)
    )
    
    # Step 6: Flag disposition for the entity
    results['flag_disposition_entity'] = np.where(
        # check for the first condition entity seller and buyer are the same and the sale index and next sale index is 1
        ((results['entity_owner_name_seller'] == results['entity_owner_name_buyer']) & (results['sale_index'] == results['next_sale_index']))
        # also check if the entity owner name is not entity
        | (results['entity_owner_name_seller'] != entity),
        # if results are true then set the flag to 1 else set it to nan
        np.nan,
        1
        )
        
    # Step 7: Flag acquisition for the entity based on sale time difference
    results['flag_acquisition_entity'] = np.where(
        (results['days_between_sales']<30 & (results['entity_owner_name_seller']== entity) & (results['entity_owner_name_buyer'] != entity)),
        np.nan,
        np.where(results['entity_owner_name_buyer']== entity,1, np.nan)
        )

    # Step 8: Aggregate acquisitions and dispositions by quarter
    final_df = (
        results
        .groupby('quarter')  # Group by quarter
        .agg(
            total_acquisitions=('flag_acquisition_entity', 'sum'),  # Rename acquisition column
            total_dispositions=('flag_disposition_entity', 'sum')   # Rename disposition column
        )
        .reset_index()  # Reset index to make 'quarter' a column
    )
    
    # Step 9: Add the entity name as a new column
    final_df['entity'] = entity
    # Step 10: Add net column to calculate the net acquisitions
    final_df['net'] = final_df['total_acquisitions'] - final_df['total_dispositions']
    
    return final_df  # Return the finnal dataframe

In [ ]:
# Process acquisition and disposition events for American Homes 4 Rent (AMH)*
# This section applies our previously defined function to analyze AMH's portfolio changes*acquisitions_dispositions_amh = process_events(sale_events_amh, 'AMH')
acquisitions_dispositions_amh = process_events(sale_events_amh, 'AMH')
acquisitions_dispositions_amh.tail(10)

In [ ]:
# Repeat for Tricon
acquisitions_dispositions_tricon = process_events(sale_events_tricon, 'TRICON')
acquisitions_dispositions_tricon.tail(10)

In [12]:
# Define the function to create the acquisitions and dispositions chart
def create_acq_disp_chart(df, path = None, market_name=market_name):
    # Convert Period to string for Plotly compatibility
    df['quarter'] = df['quarter'].astype(str)

    CHART_HEIGHT=950
    CHART_WIDTH=1800

    # Create subplot with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    # Add trace for Acquisitions
    fig.add_trace(
        go.Bar(
            x=df['quarter'],
            y=df['total_acquisitions'],
            name="Acquisitions",
            marker_color=style_config["bar1_color"]
        ),
        secondary_y=False,
    )

    # Add trace for Dispositions (negative values)
    fig.add_trace(
        go.Bar(
            x=df['quarter'],
            y=-df['total_dispositions'],
            name="Dispositions",
            marker_color=" #99ddff"
        ),
        secondary_y=False,
    )

    # Calculate y-axis ranges
    y_max = max(df['total_acquisitions'].max(), df['total_dispositions'].max()) * 1.15
    y_min = -y_max
    y_range = y_max - y_min

    # Compress net effect range
    net_max = max(abs(df['net'].max()), abs(df['net'].min()))
    compression_factor = 0.6  # Adjust this value to change compression
    net_range = y_range * compression_factor
    net_scale = net_range / (2 * net_max)

    # Scale net effect values
    scaled_net = df['net'] * net_scale

    # Add trace for Net Effect
    fig.add_trace(
        go.Scatter(
            x=df['quarter'],
            y=scaled_net,
            mode='lines+markers+text',
            name="Net Effect",
            line=dict(color='#FCC054', width=3),
            marker=dict(size=8, color='#FCC054'),
            text=df['net'].round(2),
            textposition='top center',
            textfont=dict(color='white'),
            hovertemplate='%{x}<br>Net: %{text:.2f}<extra></extra>'
        ),
        secondary_y=False,
    )

    # Update layout for dark mode and styling
    title_text = f"Investor Acquisitions, Dispositions and Net Effect".upper()
    if market_name:
        title_text = f"Acquisitions vs Dispositions For {df['entity'].iloc[0]}: {market_name}".upper()
    fig.update_layout(
        title={
            'text': title_text, # <br><span style='font-size: 16px; font-style: italic;'>{market_name}</span>",
            'y': 0.95,
            'x': 0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(size=20, color='white')
        },
        font=dict(color='white', size=14),
        plot_bgcolor="#000000",
        paper_bgcolor="#000000",
        height=CHART_HEIGHT,
        width=CHART_WIDTH,
        hovermode="x unified",
        barmode='relative',
        margin=dict(l=0, r=0, t=0, b=0),
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=0,
            xanchor="left",
            x=0
        )
    )

    # Update x-axis
    style_config["grid_color"]
    fig.update_xaxes(
        title_text="",
        showgrid=True,
        gridwidth=0.5,
        gridcolor=style_config["grid_color"],
        tickangle=45,
        dtick="M6",
        tickformat=',',
        tickfont=dict(size=12),
        linecolor='#132D59',
        linewidth=2,
        mirror=True,
        title_font=dict(family="Inter")
    )

    # Update y-axis (Acquisitions, Dispositions, and Net Effect)
    fig.update_yaxes(
        title_text="Acquisitions vs. Dispositions".upper(),
        title_font=dict(size=10),
        showgrid=True,
        gridwidth=0.5,
        gridcolor=style_config["grid_color"],
        tickfont=dict(size=12),
        title_standoff=8,
        linecolor=style_config["line_color_axis"],
        linewidth=2,
        mirror=True,
        range=[y_min + (y_min*0.1), y_max + (y_max*0.1)],
        secondary_y=False
    )

    # Add Parcl Labs watermark
    fig.add_layout_image(create_labs_logo_dict(xanchor ="right",yanchor = "bottom",x=0.93))


    fig.update_traces(
        textfont=dict(color='#FCC054'),
        texttemplate='<span style="text-shadow: -1px -1px 0 #000, 1px -1px 0 #000, -1px 1px 0 #000, 1px 1px 0 #000;">%{text:.0f}</span>',
        selector=dict(type='scatter', mode='lines+markers+text')
    )


    
    # Save the figure if a save path is provided
    save_figure(fig, save_path=path, width=CHART_WIDTH, height=CHART_HEIGHT)

    # Display the figure
    fig.show()

In [ ]:
# Create the acquisition and disposition chart for AMH
create_acq_disp_chart(acquisitions_dispositions_amh, 
                      path ='../../../images/acquisitions_vs_dispositions_amh_phoenix.png', 
                      market_name=market_name)

In [ ]:
# Repeat for Tricon
create_acq_disp_chart(acquisitions_dispositions_tricon, 
                      path ='../../../images/acquisitions_vs_dispositions_tricon_phoenix.png', 
                      market_name=market_name)

Having analyzed acquisition and disposition patterns for Tricon and AMH, we'll now examine their rental activities in the Phoenix market. This deeper analysis focuses on rental event data for properties owned by each entity.

Key aspects of the upcoming analysis:

1. Data source: Properties identified in previous steps (using parcl_property_ids)
2. Event type: RENTAL
3. Time range: 2023-01-01 to 2024-09-01
4. Entities: AMH and Tricon (analyzed separately)

This rental event data will enable us to:

- Quantify rental listing frequency
- Track rental price trends
- Compare rental strategies between entities

The following code retrieves and processes this data, preparing it for detailed rental market analysis.

In [ ]:
# get rental information for AMH
# Pass the parcl_property_ids from the search results to a list named search_results_ids to retrieve the sale events 
# for those properties.
search_results_ids_amh = amh_homes_phoenix['parcl_property_id'].tolist()

# Define the parameters we want to use in the search for property events.
property_events_parameters_amh = {
    'parcl_property_ids': search_results_ids_amh,
    'event_type': 'RENTAL',
    'entity_owner_name': 'AMH',# Specify one of the options or None
    'start_date': '2023-01-01',
    'end_date': '2024-09-01',
}

# Call the client with the list of property ids and the event_type as 'SALE' to retrieve the sale events for the properties.
# we can pass the search_params dictionary to the retrieve method to get the search results using **property_events_parameters
rent_events_amh = client.property.events.retrieve(
    **property_events_parameters_amh
    )

print(f"Found {len(rent_events_amh)} events matching the criteria.")
print(rent_events_amh.head(2))

In [ ]:
# Pass the parcl_property_ids from the search results to a list named search_results_ids to retrieve the sale events 
# for those properties.
search_results_ids_tricon = tricon_homes_phoenix['parcl_property_id'].tolist()

# Define the parameters we want to use in the search for property events.
property_events_parameters_tricon = {
    'parcl_property_ids': search_results_ids_tricon,
    'event_type': 'RENTAL',
    'entity_owner_name': 'TRICON',
     # Specify one of the options or None
    'start_date': '2023-01-01',
    'end_date': '2024-09-01',
}

# Call the client with the list of property ids and the event_type as 'SALE' to retrieve the sale events for the properties.
# we can pass the search_params dictionary to the retrieve method to get the search results using **property_events_parameters
rent_events_tricon = client.property.events.retrieve(
    **property_events_parameters_tricon
    )

print(f"Found {len(rent_events_tricon)} events matching the criteria.")
print(rent_events_tricon.head(2))

In [ ]:
# aggregate to monthly rental data
monthly_aggregated_amh = (
    rent_events_amh
    .assign(
        event_date=pd.to_datetime(rent_events_amh['event_date']),  # Convert 'event_date' to datetime
        month=lambda df: df['event_date'].dt.to_period('M')  # Create 'month' column
    )
    .groupby(['month', 'entity_owner_name'])
    .agg(
        average_price=('price', 'mean'),  # Calculate average price per group
        total_number_of_events=('event_name', 'size')  # Count the number of events per group
    )
    .reset_index()
)

# Repat for Tricon
monthly_aggregated_tricon = (
    rent_events_tricon
    .assign(
        event_date=pd.to_datetime(rent_events_tricon['event_date']),  # Convert 'event_date' to datetime
        month=lambda df: df['event_date'].dt.to_period('M')  # Create 'month' column
    )
    .groupby(['month', 'entity_owner_name'])
    .agg(
        average_price=('price', 'mean'),  # Calculate average price per group
        total_number_of_events=('event_name', 'size')  # Count the number of events per group
    )
    .reset_index()
)

# Merge both datasets and add suffixes
monthly_aggregated = pd.merge(
    monthly_aggregated_amh,
    monthly_aggregated_tricon,
    on='month',
    how='left',
    suffixes=('_amh', '_tricon')
)
monthly_aggregated['month'] = pd.PeriodIndex(monthly_aggregated['month'], freq='M')
monthly_aggregated['month'] = monthly_aggregated['month'].dt.to_timestamp()
monthly_aggregated.head()

Let's visualize those trends using a line chart to understand where the behavior of these two entities looks similar and where the are divergences. We will write this code as a function so you can re use it later without the need to type everything.

In [18]:
# define function to create chart
def create_two_series_line_chart(
    title: str,
    line_data_1: pd.DataFrame,
    series_1: str,
    line_data_2: pd.DataFrame,
    series_2: str,
    date_column: str = "date",
    save_path: str = None,
    yaxis_title: str = "Y-Axis",
    height=675,
    width=1200,
    yaxis_prefix: str = '$',
    style_config=None,  # Default is None, will fall back on default_style_config
):
    """
    Creates a dual-series line chart with customizable style options and saves or displays it.

    Parameters:
    ----------
    title : str
        The title of the chart.
    line_data_1 : pd.DataFrame
        The first dataframe containing the data for the first line series.
    series_1 : str
        The column name in line_data_1 representing the values for the first line series.
    line_data_2 : pd.DataFrame
        The second dataframe containing the data for the second line series.
    series_2 : str
        The column name in line_data_2 representing the values for the second line series.
    date_column : str, optional
        The name of the column in both dataframes representing dates (default is "date").
    save_path : str, optional
        The file path where the figure should be saved (default is None, meaning the chart won't be saved).
    yaxis_title : str, optional
        The title of the Y-axis (default is "Y-Axis").
    height : int, optional
        The height of the figure in pixels (default is 675).
    width : int, optional
        The width of the figure in pixels (default is 1200).
    yaxis_prefix : str, optional
        The prefix for the Y-axis labels (default is '$', used for financial data).
    style_config : dict, optional
        A dictionary for customizing the appearance of the chart, such as colors and fonts. 
        If None is provided, it uses the default_style_config (default is None).

    Returns:
    -------
    None
        Displays the plot using Plotly and optionally saves the figure if a save_path is provided.

    Notes:
    ------
    - The chart will display two time series, each as a line plot with markers.
    - The `style_config` allows for extensive customization, such as background colors, font settings, and marker styles.
    """
    # If no style_config is provided, use the default
    if style_config is None:
        style_config = default_style_config

    fig = go.Figure()

    # Ensure the date column is in a datetime format (if it's not already)
    line_data_1[date_column] = pd.to_datetime(line_data_1[date_column])
    line_data_2[date_column] = pd.to_datetime(line_data_2[date_column])

    # Add the first time series as a line plot
    fig.add_trace(
        go.Scatter(
            x=line_data_1[date_column],
            y=line_data_1[series_1],
            mode="lines+markers",
            line=dict(
                width=style_config["line_width"],
                color=style_config["line_color"]
            ),
            marker=dict(
                size=style_config["marker_size"],
                color=style_config["marker_color"],
                line=dict(width=1, color=style_config["marker_outline_color"]),
            ),
            name=series_1
        )
    )

    # Add the second time series as a line plot
    fig.add_trace(
        go.Scatter(
            x=line_data_2[date_column],
            y=line_data_2[series_2],
            mode="lines+markers",
            line=dict(
                width=style_config["line_width"],
                color=style_config["bar1_color"]  # Using bar1_color for the second line
            ),
            marker=dict(
                size=style_config["marker_size"],
                color=style_config["bar1_color"],
                line=dict(width=1, color=style_config["marker_outline_color"]),
            ),
            name=series_2
        )
    )
    style_config["title_font"]['size']=20
    # Update layout to match the styling conventions
    fig.update_layout(
        margin=dict(l=40, r=40, t=80, b=40),
        height=height,
        width=width,
        title={
            "text": title,
            "y": 0.95,
            "x": 0.5,
            "xanchor": "center",
            "yanchor": "top",
            "font": style_config["title_font"],
            
        },
        plot_bgcolor=style_config["background_color"],
        paper_bgcolor=style_config["background_color"],
        font=dict(color=style_config["font_color"]),
        xaxis=dict(
            title_text="",
            tickformat='%b %Y',  # Formatting for monthly data (e.g., Jan 2021)
            dtick="M2",  # Show every two months
            showgrid=style_config["showgrid"],
            gridwidth=style_config["gridwidth"],
            gridcolor=style_config["grid_color"],
            tickangle=style_config["tick_angle"],
            tickfont=style_config["axis_font"],
            linecolor=style_config["line_color_axis"],
            linewidth=style_config["linewidth"],
            titlefont=style_config["title_font_axis"],
        ),
        yaxis=dict(
            title_text=yaxis_title,
            showgrid=style_config["showgrid"],
            gridwidth=style_config["gridwidth"],
            gridcolor=style_config["grid_color"],
            tickfont=style_config["axis_font"],
            tickformat=yaxis_prefix,  # Format y-axis with comma for thousands
            zeroline=False,
            linecolor=style_config["line_color_axis"],
            linewidth=style_config["linewidth"],
            titlefont=style_config["title_font_axis"],
        ),
        hovermode="x unified",
        hoverlabel=dict(
            bgcolor=style_config["hover_bg_color"],
            font_size=style_config["hover_font_size"],
            font_family=style_config["hover_font_family"],
            font_color=style_config["hover_font_color"],
        ),
        legend=dict(
            x=style_config["legend_x"],
            y=style_config["legend_y"],
            xanchor=style_config["legend_xanchor"],
            yanchor=style_config["legend_yanchor"],
            font=style_config["legend_font"],
            bgcolor="rgba(0, 0, 0, 0)",
        )
    )

    # Add a logo to the chart (if applicable)
    fig.add_layout_image(create_labs_logo_dict())

    # Save the figure if a save path is provided
    save_figure(fig, save_path=save_path, width=width, height=height)

    # Show the figure
    fig.show()


In [ ]:

# Example call to the function
style_config['tick_prefix'] = ''
create_two_series_line_chart(
    title=f"Average Monthly Rental Price Tricon and AMH in {market_name}".upper(),
    line_data_1=monthly_aggregated,
    series_1="average_price_amh",
    line_data_2=monthly_aggregated,
    series_2="average_price_tricon",
    date_column="month",
    yaxis_prefix='$,',
    yaxis_title="Average Monthly Rental Price",
    height=SIZE_CONFIG['x']['height'],
    width=SIZE_CONFIG['x']['width'],
    style_config=style_config,
    save_path='../../../images/entity_monthly_rental_price_phoenix.png'
)


In [ ]:

# Repeat for total rental events for AMH and Tricon
style_config['tick_prefix'] = '$'
create_two_series_line_chart(
    title=f"Total Monthly Rental Events AMH and Tricon in {market_name}".upper(),
    line_data_1=monthly_aggregated,
    series_1="total_number_of_events_amh",
    line_data_2=monthly_aggregated,
    series_2="total_number_of_events_tricon",
    date_column="month",
    yaxis_prefix=',',
    yaxis_title="Total Monthly Rental Events",
    height=SIZE_CONFIG['x']['height'],
    width=SIZE_CONFIG['x']['width'],
    style_config=style_config,
    save_path='../../../images/entity_total_rental_events_phoenix.png'
)
